<a href="https://colab.research.google.com/github/ekremerakin/make_google_colab_faster/blob/master/Make_Google_Colab_25x_Faster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to Google Drive.

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Copying the zipped dataset into the local directory of Google Colab. Change the `fill_the_file_name` with your own file path.

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/fill_the_file_name.zip", 'r')
zip_ref.extractall("/content/dataset")
zip_ref.close()

# Training performance check

I used a dataset called LFW to test the performance difference. LFW is a face recognition dataset and includes over 5k individuals. You can download the dataset and try out your own. Also, you can try out with your own dataset here but you might need to do some adjustments.

Change the `fill_the_dataset_dir` or other file directories depending on your needs.

In [ ]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

# Reading from Google Colab
input_path = "/content/dataset/"

# Reading from Google Drive
# input_path = '/content/drive/MyDrive/fill_the_dataset_dir'

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])
}

image_datasets = {
    'train':
    datasets.ImageFolder(input_path, data_transforms['train'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0)
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.resnet18(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
               nn.Linear(512, 5749)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

from torchsummary import summary

summary(model, (3, 112, 112))

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [ ]:
import time as t

start_time = t.time()
model_trained = train_model(model, criterion, optimizer, num_epochs=1)
end_time = t.time()

print(end_time-start_time)